# CSE 6240 - Web Search & Text Mining - Homework 3

In [1]:
# __author__ = 'Bhanu Verma'
# GTid = '903151012'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import linalg
import os, os.path
from scipy.sparse import lil_matrix,linalg
%matplotlib inline

In [20]:
def get_matrix_dimensions():
    doc_size = 0
    vocab_size = 0
    with open(filepath, "r") as f:
        count = 0
        for line in f:
            line = line.rstrip('\n')
            if count==0:
                doc_size = int(line)
            elif count==1:
                vocab_size = int(line)
            else:
                break
                
            count+=1
    
    f.close()
    
    return (doc_size,vocab_size)

In [21]:
def populate_matrix():
    with open(filepath, "r") as f:
        count = 0
        for line in f:
            line = line.rstrip('\n')
            if count > 2:
                arr = line.split()
                i = int(arr[0])-1
                j = int(a)
                doc_word_mat[int(arr[0]-1),int(arr[1])] = int(arr[2])
            count+=1
    f.close()

In [23]:
filepath = 'Data/docword.kos.txt'
size_tup = get_matrix_dimensions()
print size_tup

(3430, 6906)


In [24]:
doc_word_mat = lil_matrix((size_tup[0], size_tup[1]), dtype=np.int32)
populate_matrix()

IndexError: column index (6906) out of bounds